# Описание ДЗ1.

На основе семинара 1 предложите 2 метода улучшения построения эмбеддингов вопросов на основе word vectors.

За задание можно получить максимум 10 баллов.

За каждый метод можно получить максимум 5 баллов.
Разбалловка:
*   Воспроизводимость и читабельность кода - 1 балл.
*   Корректность метода - 1 балл.
*   Описание метода в техническом отчете - 2 балла.
*   Иновационность - 1 балл.


# 1. Информация о сабмите

Щербаков Игорь Андреевич

# 2. Технический отчет

***Введите сюда** подробное описание предложенных методов и экспериментов, с помощью которых вы пришли именно к выбору этих методов. НЕ вставляйте код в эту часть. Описание должно состоять минимум из 2-4 абзацев и содержать следующее: тип модели, параметры, как вы выбрали параметры, какие дальнейшие модификации готовых решений и т.д. вы использовали. Сюда можно включить, например, некоторые хитрости вашей предварительной обработки, описание моделей и мотивацию их использования, описание деталей процесса обучения. Если нужно, вставьте сюда графики, математические формулы.*

*Балл за «инновационность» будет присваиваться на основе содержания этой части. Если ваше отличие от бейзлайна это просто почистить тексты от стоп-слов или поменять одну базовую модель для построения word embeddings на другую, этот балл будет 0. Пробуйте разные подходы, модели, экспериментируйте с предварительной обработкой, параметрами и т. д. Можно переопределить уже существующий подход. Это нормально, что некоторые из ваших экспериментов не сработали так, как вы ожидали. Покажите нам, что вы проявили творческий подход и провели несколько экспериментов.*

<h2>Возможные шаги</h2>
Какие шаги были бы возможны:
<ul> 1. Предобработка данных х</ul>
<ul> 2. Подбор модели</ul>
<ul> 3. Подбор параметров модели</ul>
<ul> 4. Эксперименты с эмбедингами слов (добавление доп размерностей с новой информацией)</ul>
<ul> 5. Изменеиение способа получения представления фразы на основе представлений слов</ul>
<ul> 6. Модификация алгоритмов поиска похожих: повышение качества и скорости</ul>


Подбор разных моделей и их параметров, кажется весьма обыденным, поэтому в этом ноутбуке рассмотрим следующщие подходы: предобработку данных, дополнительные размерности в эмбединг слова, представление фразы на основе слов.

<h2>Предпринятые шаги</h2>
<h3>Предобработка</h3>
В качестве дополнитльной предобработки были добавлены:
<li>Удаление стоп слов
<li> Алгоритм поиска похожих заменн]н на реализацию в FAISS, что позволило многократно ускорить поиск и повысить его релевантность 



# 3. *Code*

*Введите сюда весь код, использованный для получения результатов. Добавьте несколько комментариев и подразделов для навигации по вашему решению.*

*В этой части вам предстоит самостоятельно разработать решение задачи и предоставить воспроизводимый код:*
- *Использование Python 3;*
- *Содержит код для установки всех зависимостей;*
- *Содержит код для загрузки всех используемых наборов данных*;
- *Содержит код для воспроизведения ваших результатов (другими словами, если проверяющий загрузит ваш блокнот, он сможет выполнить код по ячейкам и получить результаты эксперимента, как описано в разделе методологии)*.


*В результате ваш код будет оценен по следующим критериям:*
- ***Читаемость**: ваш код должен быть хорошо структурирован, желательно с указанием частей вашего подхода (предварительная обработка, обучение модели, тестирование модели и т. д.).*
- ***Воспроизводимость**: ваш код должен воспроизводиться без ошибок в режиме «Выполнить все» (получение экспериментальной части).*


## 3.1 Импорт библиотек

In [1]:
#Установка бибилотек в окружение
!pip install -q -r requirements.txt

In [2]:
import numpy as np
from nltk.tokenize import WordPunctTokenizer
import gensim.downloader as api

## 3.2 Загрузка данных

In [69]:
# download the data:

#!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
#!wget https://yadi.sk/i/BPQrUu1NaTduEw -O ./quora.txt

!wget https://raw.githubusercontent.com/Falconwatch/llm_course/main/HW1/quora.txt -O ./quora.txt

data = np.array(list(open("./quora.txt", encoding="utf-8")))

--2024-05-26 10:25:01--  https://raw.githubusercontent.com/Falconwatch/llm_course/main/HW1/quora.txt
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 33813903 (32M) [text/plain]
Сохранение в: «./quora.txt»

./quora.txt         100%[===================>]  32,25M  10,4MB/s    за 3,1s    

2024-05-26 10:25:04 (10,4 MB/s) - «./quora.txt» сохранён [33813903/33813903]



## 3.3. Модель

In [70]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /Users/igor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [71]:
STOP_WORDS = set(stopwords.words('english')) 
TOKENIZER = WordPunctTokenizer()
MODEL = api.load('glove-twitter-100')

#model = FastText.load_fasttext_format('wiki.simple')

In [72]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    vector = np.zeros([MODEL.vector_size], dtype='float32')
    word_vectors = []

    data_tok = TOKENIZER.tokenize(phrase.lower())
    word_vectors = np.array([MODEL.get_vector(word) for word in data_tok 
                             if (word in MODEL.key_to_index.keys()) and (word not in STOP_WORDS)])

    if word_vectors.size != 0:
        vector = np.mean(word_vectors, axis=0)

    return vector

In [73]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

### Складываем наш датасет в базу

In [93]:
import faiss
class MyDb:
    def __init__(self, sentences, sentence_embedings):
        self._dim = len(sentence_embedings[0])
        self._faiss = faiss.IndexFlatL2(self._dim )
        self._faiss.add(sentence_embedings)
        self._sentences = sentences

    def get_similar(self, query_embeddings, k=5):
        result = []
        if (len(query_embeddings) == 0) or (len(query_embeddings[0])==0):
            raise Exception("query_embeddings должно быть списком векторов-запросов для поиска похожих")
        d, i = self._faiss.search(x= query_embeddings, k = k)
        
        for r in i:
            r_sent = self._sentences[i]
            result.append(r_sent)
        return result

In [101]:
MDB = MyDb(data, data_vectors)

## 3.4. Применение модели

In [109]:
def find_nearest(query, k=10):
    emb = get_phrase_embedding(query)
    result = MDB.get_similar(np.array([emb]), k)
    return result

In [112]:
find_nearest("Who is Trump")

[array([['Will Donald Trump destroy Donald Trump?\n',
         'How did Melania Trump meet Donald Trump?\n',
         'Is Donald trump a populist?\n',
         'Should the NRA withdraw its endorsement of Donald Trump?\n',
         'Why is Quora biased agaist Donald Trump.?\n',
         'Donald Trump: Does Donald Trump have misogynistic views?\n',
         'Is Donald Trump a blowhard?\n',
         'What did Ivanka Trump do at Wharton?\n',
         'Is Donald Trump a “Teflon candidate”?\n',
         'Why is Quora against Donald Trump?\n']], dtype='<U1170')]